In [4]:
import torchvision.transforms as transforms
import torchvision
import torch

transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])

#train_data_path = 'relative path of training data set'
train_data_path = './horse-or-human/train'
trainset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=1)  


validation_data_path = './horse-or-human/validation'
valset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
valloader = torch.utils.data.DataLoader(valset, batch_size=1, shuffle=True, num_workers=1) 


'''
for i, data in enumerate(trainloader):
    
    # inputs is the image
    # labels is the class of the image
    trnInputs, trnLabels = data

    # if you don't change the image size, it will be [batch_size, 1, 100, 100]
    # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]
    
    #make inputs [1027, 1, 100, 100] to [10000,1027]
    
    trnInputs = trnInputs.view(1027, 10000)
    trnInputs = trnInputs.T
    trnInputs = trnInputs.double()
    trnLabels = trnLabels.double()

'''
'''
# load validation images of the batch size for every iteration
for i, data in enumerate(valloader):

    # inputs is the image
    # labels is the class of the image
    valInputs, valLabels = data

    # if you don't change the image size, it will be [batch_size, 1, 100, 100]
    # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]
    #make [256, 1, 100, 100] to [10000,256]
    valInputs = valInputs.view(256, 10000)
    valInputs = valInputs.T
    valInputs = valInputs.double()
    valLabels = valLabels.double()


totalTrnNum = trnLabels.shape[0]
totalValNum = valLabels.shape[0]
'''

"\n# load validation images of the batch size for every iteration\nfor i, data in enumerate(valloader):\n\n    # inputs is the image\n    # labels is the class of the image\n    valInputs, valLabels = data\n\n    # if you don't change the image size, it will be [batch_size, 1, 100, 100]\n    # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]\n    #make [256, 1, 100, 100] to [10000,256]\n    valInputs = valInputs.view(256, 10000)\n    valInputs = valInputs.T\n    valInputs = valInputs.double()\n    valLabels = valLabels.double()\n\n\ntotalTrnNum = trnLabels.shape[0]\ntotalValNum = valLabels.shape[0]\n"

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import math

class Linear(nn.Module):

    def __init__(self, num_classes=2):

        super(Linear, self).__init__()

        self.number_class   = num_classes

        _size_image     = 100* 100
        _num1           = 50
        _num2           = 50
        
        self.fc1        = nn.Linear(_size_image, _num1, bias=True)
        self.fc2        = nn.Linear(_num1, _num2, bias=True)
        self.fc3        = nn.Linear(_num2, num_classes, bias=True)

        self.fc_layer1  = nn.Sequential(self.fc1, nn.ReLU(True))
        self.fc_layer2  = nn.Sequential(self.fc2, nn.ReLU(True))
        self.fc_layer3  = nn.Sequential(self.fc3, nn.ReLU(True))
        
        self.classifier = nn.Sequential(self.fc_layer1, self.fc_layer2, self.fc_layer3)
        
        self._initialize_weight()        
        
    def _initialize_weight(self):

        for m in self.modules():
            

            if type(m) == nn.Linear:
            
                n = m.in_features
                m.weight.data.uniform_(- 1.0 / math.sqrt(n), 1.0 / math.sqrt(n))
                if m.bias is not None:

                    m.bias.data.zero_()
                    

    def forward(self, x):

        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x


In [ ]:
# -----------------------------------------------------------------------------
# import packages
# -----------------------------------------------------------------------------
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import argparse
import sys
import os
import numpy as np
import time
import datetime 
import csv
import configparser
import argparse
import platform

from torchvision import datasets, transforms
from torch.autograd import Variable
from random import shuffle

# -----------------------------------------------------------------------------
# load dataset
# -----------------------------------------------------------------------------

#set_train   = 
#set_test    = 

num_classes = 2

# -----------------------------------------------------------------------------
# load neural network model
# -----------------------------------------------------------------------------


model = Linear(num_classes=num_classes)

# -----------------------------------------------------------------------------
# Set the flag for using cuda
# -----------------------------------------------------------------------------

bCuda = 0

if bCuda:
 
    model.cuda()

# -----------------------------------------------------------------------------
# optimization algorithm
# -----------------------------------------------------------------------------

optimizer   = optim.SGD(model.parameters(), lr = 0.001)
objective   = nn.CrossEntropyLoss()

# -----------------------------------------------------------------------------
# function for training the model
# -----------------------------------------------------------------------------

def train():

    # print('train the model at given epoch')

    loss_train          = []

    model.train()

    for idx_batch, (data, target) in enumerate(trainloader):

        if bCuda:
        
            data, target    = data.cuda(), target.cuda()

        data, target    = Variable(data), Variable(target)

        optimizer.zero_grad()

        output  = model(data)
        loss    = objective(output, target)

        loss.backward()
        optimizer.step()

        loss_train_batch    = loss.item() / len(data)
        loss_train.append(loss_train_batch)
        
    loss_train_mean     = np.mean(loss_train)
    loss_train_std      = np.std(loss_train)

    return {'loss_train_mean': loss_train_mean, 'loss_train_std': loss_train_std}

# -----------------------------------------------------------------------------
# function for testing the model
# -----------------------------------------------------------------------------

def test():

    # print('test the model at given epoch')

    accuracy_test   = []
    loss_test       = 0
    correct         = 0

    model.eval()

    for idx_batch, (data, target) in enumerate(valloader):

        if bCuda:
        
            data, target    = data.cuda(), target.cuda()

        data, target    = Variable(data), Variable(target)

        output  = model(data)
        loss    = objective(output, target)

        loss_test   += loss.item()
        pred        = output.data.max(1)[1]
        correct     += pred.eq(target.data.view_as(pred)).cpu().sum()

    loss_test       = loss_test / len(valloader.dataset)
    accuracy_test   = 100. * float(correct) / len(valloader.dataset)

    return {'loss_test': loss_test, 'accuracy_test': accuracy_test}

# -----------------------------------------------------------------------------
# iteration for the epoch
# -----------------------------------------------------------------------------


loss_train_mean =[]
loss_train_std = []
accuracy_test = []
loss_test = []

for e in range(1000):
    result_train    = train()
    result_test     = test()

    loss_train_mean.append(result_train['loss_train_mean'])
    loss_train_std.append(result_train['loss_train_std'])
    loss_test.append(result_test['loss_test'])
    accuracy_test.append(result_test['accuracy_test'])
    
    print("trainloss = ", loss_train_mean[e], " testloss = ", loss_test[e], " accuracy_test = ", accuracy_test[e])


trainloss =  0.008582344530808804  testloss =  0.6884284629486501  accuracy_test =  50.78125
trainloss =  0.008564282418902875  testloss =  0.6873050518333912  accuracy_test =  50.390625
trainloss =  0.00858574021544922  testloss =  0.6849511361215264  accuracy_test =  50.390625
trainloss =  0.008538078246293244  testloss =  0.6826330353505909  accuracy_test =  51.953125
trainloss =  0.008512135380847687  testloss =  0.6820733994245529  accuracy_test =  51.171875
trainloss =  0.008513280279708631  testloss =  0.6769618424586952  accuracy_test =  55.078125
trainloss =  0.008486718033700678  testloss =  0.6764169645030051  accuracy_test =  52.734375
trainloss =  0.008461034851844866  testloss =  0.6730424123816192  accuracy_test =  54.6875
trainloss =  0.008441509439889028  testloss =  0.6695729536004364  accuracy_test =  58.59375
trainloss =  0.008423548645844764  testloss =  0.6679292193148285  accuracy_test =  55.078125
trainloss =  0.008349922325474645  testloss =  0.671322544105351 

trainloss =  0.006689561205078857  testloss =  0.4187037630472332  accuracy_test =  85.546875
trainloss =  0.006502173538962598  testloss =  0.4065837364178151  accuracy_test =  87.5
trainloss =  0.006552921540608711  testloss =  0.40675017423927784  accuracy_test =  86.71875
trainloss =  0.006323927345902028  testloss =  0.4039199464023113  accuracy_test =  86.71875
trainloss =  0.006600756924160403  testloss =  0.401872486108914  accuracy_test =  87.109375
trainloss =  0.006379756961607371  testloss =  0.3971766645554453  accuracy_test =  87.5
trainloss =  0.00647247344957859  testloss =  0.3942514779046178  accuracy_test =  87.109375
trainloss =  0.006769933948412487  testloss =  0.3940361423883587  accuracy_test =  86.71875
trainloss =  0.006384113469107785  testloss =  0.41827002074569464  accuracy_test =  82.8125
trainloss =  0.006153298688657356  testloss =  0.38629771303385496  accuracy_test =  87.890625
trainloss =  0.006233390792451724  testloss =  0.3874779643956572  accurac

trainloss =  0.005282910251456881  testloss =  0.28523301612585783  accuracy_test =  87.5
trainloss =  0.005205739589050563  testloss =  0.30791020346805453  accuracy_test =  86.328125
trainloss =  0.0054313212422409446  testloss =  0.37882918398827314  accuracy_test =  81.25
trainloss =  0.0052836001691994846  testloss =  0.29003274627029896  accuracy_test =  88.28125
trainloss =  0.0049875962573671185  testloss =  0.3069315254688263  accuracy_test =  86.328125
trainloss =  0.005369792316698466  testloss =  0.29511652421206236  accuracy_test =  88.28125
trainloss =  0.004829514648777868  testloss =  0.2891686079092324  accuracy_test =  88.28125
trainloss =  0.004953745488165204  testloss =  0.302479749545455  accuracy_test =  87.109375
trainloss =  0.005013805488343994  testloss =  0.31563921459019184  accuracy_test =  85.15625
trainloss =  0.0045112504782500095  testloss =  0.3009625496342778  accuracy_test =  87.109375
trainloss =  0.005032378854574981  testloss =  0.303804179187864